In [ ]:
!pip uninstall -y torch torchvision torchaudio


!pip install -q --no-cache-dir \
torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121 \
--index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 MB 260.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 221.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 153.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 314.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 410.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 347.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 235.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q --no-cache-dir \
  pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-2.4.1+cu121.html

!pip install -q --no-cache-dir torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.6/989.6 kB 233.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
!pip install relbench

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
pip install -U "autogluon.tabular[all]"

In [ ]:
import torch
import torch_geometric
from torch_geometric.loader import NeighborLoader

print("torch:", torch.__version__)
print("pyg:", torch_geometric.__version__)
print("NeighborLoader import OK")

torch: 2.4.1+cu121
pyg: 2.7.0
NeighborLoader import OK


In [ ]:
import numpy as np
import pandas as pd

import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from relbench.datasets import get_dataset

dataset = get_dataset(name="rel-trial", download=True)

100%|████████████████████████████████████████| 574M/574M [00:00<00:00, 970GB/s]
Unzipping contents of '/root/.cache/relbench/rel-trial/db.zip' to '/root/.cache/relbench/rel-trial/.'


In [ ]:
from relbench.tasks import get_task

db = dataset.get_db()

table = db.table_dict["studies"]
task = get_task("rel-trial", "study-outcome", download=True)

train_task = task.get_table("train").df
val_task = task.get_table("val").df
test_task = task.get_table("test").df

## learning table

df_train = train_task.merge(table.df, how="left")
df_val = val_task.merge(table.df, how="left")
df_test = test_task.merge(table.df, how="left")

Loading Database object from /root/.cache/relbench/rel-trial/db...
Done in 16.05 seconds.


100%|█████████████████████████████████████| 78.9k/78.9k [00:00<00:00, 18.6MB/s]
Unzipping contents of '/root/.cache/relbench/rel-trial/tasks/study-outcome.zip' to '/root/.cache/relbench/rel-trial/tasks/.'


# Build the graph

In [ ]:
from typing import Any, Collection, Dict, List, Optional, Tuple, Union
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import HeteroData

def df_to_hetero_for_task(
    df: pd.DataFrame,
    *,
    features: List[str],
    task_col: Optional[str] = None,     # <-- change
    row_encoder=None,
    row_node_type: str = "row",
    feature_node_prefix: str = "",
    include_missing: bool = True,
    missing_token: str = "__MISSING__",
    oov_token: str = "__OOV__",
    add_reverse_edges: bool = True,
    store_value_strings: bool = False,
    x_dtype: torch.dtype = torch.float32,
    y_dtype: torch.dtype = torch.float32,
    value2idx_in: Optional[Dict[str, Dict[Any, int]]] = None,
    use_one_hot: Optional[Union[bool, Collection[str]]] = None,
    one_hot_max_size: int = 5000,
) -> Tuple[HeteroData, Dict[str, Dict[Any, int]]]:

    # If task_col is provided, require it. If not, build unlabeled graph.
    if task_col is not None and task_col not in df.columns:
        raise KeyError(f"task_col='{task_col}' not in df.columns.")

    for f in features:
        if f not in df.columns:
            raise KeyError(f"Feature '{f}' not in df.columns.")

    if use_one_hot is True:
        one_hot_feats = set(features)
    elif use_one_hot:
        one_hot_feats = set(use_one_hot)
    else:
        one_hot_feats = set()

    data = HeteroData()
    n = len(df)

    data[row_node_type].num_nodes = n

    # Only attach labels when available
    if task_col is not None:
        data[row_node_type].y = torch.tensor(
            df[task_col].to_numpy(), dtype=y_dtype
        ).view(-1)

    if row_encoder is not None:
        X, _names = row_encoder.transform(df)
        data[row_node_type].x = torch.tensor(X, dtype=x_dtype)

    row_idx = np.arange(n, dtype=np.int64)

    value2idx: Dict[str, Dict[Any, int]] = {} if value2idx_in is None else value2idx_in

    for feat in features:
        feat_type = f"{feature_node_prefix}{feat}"
        rel_type = f"has_{feat}"
        rev_rel_type = f"rev_{rel_type}"

        col = df[feat]
        if include_missing:
            col = col.where(~col.isna(), other=missing_token)

        if value2idx_in is None:
            uniques = list(pd.unique(col if include_missing else df[feat].dropna()))
            if include_missing and missing_token not in uniques:
                uniques.append(missing_token)
            if oov_token not in uniques:
                uniques.append(oov_token)
            v2i = {v: i for i, v in enumerate(uniques)}
            value2idx[feat] = v2i
        else:
            v2i = value2idx_in[feat]
            if oov_token not in v2i:
                raise ValueError(f"oov_token='{oov_token}' missing in value2idx_in[{feat!r}]")

        num_vals = len(v2i)
        data[feat_type].num_nodes = num_vals

        if (feat in one_hot_feats) and (num_vals <= one_hot_max_size):
            data[feat_type].x = torch.eye(num_vals, dtype=x_dtype)

        if store_value_strings and value2idx_in is None:
            data[feat_type].value = list(v2i.keys())

        oov_idx = v2i[oov_token]
        vals = col.to_numpy()
        dst = np.fromiter((v2i.get(v, oov_idx) for v in vals), dtype=np.int64, count=len(vals))

        edge_index = torch.tensor(np.vstack([row_idx, dst]), dtype=torch.long)
        data[(row_node_type, rel_type, feat_type)].edge_index = edge_index

        if add_reverse_edges:
            data[(feat_type, rev_rel_type, row_node_type)].edge_index = edge_index.flip(0).contiguous()

    return data, value2idx


In [ ]:
tasks = ['outcome']

features = [
        "fdaaa801_violation",
        "is_unapproved_device",
        "is_ppsd",
        "is_fda_regulated_device",
        "enrollment_type",
        "has_dmc",
        "is_us_export",
        "is_fda_regulated_drug",
        "plan_to_share_ipd",
        "biospec_retention",
        "study_type",
        "source_class",
        "phase",
    ] # features to encode as nodes

task_col = "outcome"
data_train, v2i_train = df_to_hetero_for_task(
    df_train,
    features=features,
    task_col=task_col,
    row_encoder=None,          # we will attach row.x ourselves
    add_reverse_edges=True,
    include_missing=True,
    missing_token="__MISSING__",
    oov_token="__OOV__",       # useful for missing values
    value2idx_in=None,         # IMPORTANT: per-split vocab
    use_one_hot=None,
)

data_val, v2i_val = df_to_hetero_for_task(
    df_val,
    features=features,
    task_col=task_col,
    row_encoder=None,
    add_reverse_edges=True,
    include_missing=True,
    missing_token="__MISSING__",
    oov_token="__OOV__",
    value2idx_in=None,         # IMPORTANT: per-split vocab
    use_one_hot=None,
)



data_test, v2i_test = df_to_hetero_for_task(
    df_test,
    features=features,
    row_encoder=None,
    add_reverse_edges=True,
    include_missing=True,
    missing_token="__MISSING__",
    oov_token="__OOV__",
    value2idx_in=None,         # IMPORTANT: per-split vocab
    use_one_hot=None,
)


# Preprocess node level row fearures

In [ ]:
from typing import List, Optional, Union

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


class RowFeaturePreprocessor:
    def __init__(
        self,
        *,
        text_cols: Optional[Union[str, List[str]]] = None,
        numeric_cols: List[str],
        categorical_cols: Optional[List[str]] = None,
        max_tfidf_features: int = 128,
        min_df: int = 2,
        lowercase: bool = True,
        cat_missing_token: str = "__MISSING__",
    ):
        # normalize text_cols
        if isinstance(text_cols, str):
            text_cols = [text_cols]

        self.text_cols = text_cols or []
        self.numeric_cols = numeric_cols
        self.categorical_cols = categorical_cols or []

        self.max_tfidf_features = max_tfidf_features
        self.min_df = min_df
        self.lowercase = lowercase
        self.cat_missing_token = cat_missing_token

        self._tfidf = None
        self._scaler = None
        self._ohe = None
        self._fitted = False

    def fit(self, df: pd.DataFrame):
        # ---- numeric ----
        if self.numeric_cols:
            X_num = (
                df[self.numeric_cols]
                .apply(pd.to_numeric, errors="coerce")
                .fillna(0.0)
                .to_numpy(dtype=np.float32)
            )
            self._scaler = StandardScaler()
            self._scaler.fit(X_num)

        # ---- categorical ----
        if self.categorical_cols:
            X_cat = (
                df[self.categorical_cols]
                .astype("object")
                .where(~df[self.categorical_cols].isna(), self.cat_missing_token)
            )
            self._ohe = OneHotEncoder(
                handle_unknown="ignore",
                sparse_output=False,
                dtype=np.float32,
            )
            self._ohe.fit(X_cat)

        # ---- text (single TF-IDF over concatenated columns) ----
        if self.text_cols:
            text = self._combine_text_cols(df)
            self._tfidf = TfidfVectorizer(
                max_features=self.max_tfidf_features,
                min_df=self.min_df,
                lowercase=self.lowercase,
                ngram_range=(1, 2),
            )
            self._tfidf.fit(text)

        self._fitted = True
        return self

    def transform(self, df: pd.DataFrame) -> np.ndarray:
        if not self._fitted:
            raise RuntimeError("RowFeaturePreprocessor must be fitted first")

        parts = []

        # numeric
        if self.numeric_cols:
            X_num = (
                df[self.numeric_cols]
                .apply(pd.to_numeric, errors="coerce")
                .fillna(0.0)
                .to_numpy(dtype=np.float32)
            )
            X_num = self._scaler.transform(X_num)
            parts.append(X_num)

        # categorical
        if self.categorical_cols:
            X_cat = (
                df[self.categorical_cols]
                .astype("object")
                .where(~df[self.categorical_cols].isna(), self.cat_missing_token)
            )
            X_cat = self._ohe.transform(X_cat)
            parts.append(X_cat)

        # text
        if self.text_cols:
            text = self._combine_text_cols(df)
            X_text = self._tfidf.transform(text).toarray().astype(np.float32)
            parts.append(X_text)

        if not parts:
            return np.zeros((len(df), 0), dtype=np.float32)

        return np.hstack(parts)

    def _combine_text_cols(self, df: pd.DataFrame) -> pd.Series:
        # Safe concatenation with missing handling
        s = (
            df[self.text_cols]
            .fillna("")
            .astype(str)
        )
        return s.apply(lambda r: " ".join(v for v in r.values if v), axis=1)

    @property
    def output_dim(self) -> int:
        if not self._fitted:
            raise RuntimeError("Not fitted")

        dim = 0
        if self.numeric_cols:
            dim += len(self.numeric_cols)
        if self.categorical_cols:
            dim += sum(len(cats) for cats in self._ohe.categories_)
        if self.text_cols:
            dim += len(self._tfidf.get_feature_names_out())
        return dim

In [ ]:
NUMERIC_COLS = [
"target_duration",
"enrollment",
"number_of_arms",
"number_of_groups",
]

CATEGORICAL_COLS = [
"study_type",
"phase",
"enrollment_type",
"has_dmc",
"is_fda_regulated_drug",
"is_fda_regulated_device",
"is_unapproved_device",
"is_us_export",
"biospec_retention",
"source_class",
"fdaaa801_violation",
"plan_to_share_ipd",
]

TEXT_COLS = [
"acronym",
"baseline_population",
"brief_title",
"official_title",
"source",
"is_ppsd",
"biospec_description",
"baseline_type_units_analyzed",
"detailed_descriptions",
"brief_summaries",
]


pp = RowFeaturePreprocessor(
    text_cols=TEXT_COLS,
    numeric_cols=NUMERIC_COLS,
    categorical_cols=CATEGORICAL_COLS,
    max_tfidf_features=256,   # hyperparameter, maybe change?
).fit(df_train)

data_train["row"].x = torch.tensor(pp.transform(df_train), dtype=torch.float32)
data_val["row"].x   = torch.tensor(pp.transform(df_val),   dtype=torch.float32)
data_test["row"].x  = torch.tensor(pp.transform(df_test),  dtype=torch.float32)

print("row.x dim:", data_train["row"].x.shape[1])


row.x dim: 305


# GNN

In [ ]:
from typing import Dict, List, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv


class WideDeepHeteroSAGE(nn.Module):
    def __init__(
        self,
        metadata: Tuple[List[str], List[Tuple[str, str, str]]],
        *,
        num_nodes_by_type: Dict[str, int],
        row_in_dim: int,
        row_node_type: str = "row",
        hidden_channels: int = 128,
        num_layers: int = 1,
        dropout: float = 0.2,
        sage_aggr: str = "mean",
        mlp_hidden: int = 256,
        use_layernorm: bool = True,
    ):
        super().__init__()
        self.node_types, self.edge_types = metadata
        self.row_node_type = row_node_type
        self.hidden = hidden_channels
        self.num_layers = num_layers
        self.dropout = dropout

        # --- Wide path: row-only MLP ---
        self.row_mlp = nn.Sequential(
            nn.Linear(row_in_dim, mlp_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, hidden_channels),
        )

        # --- Graph path init for row nodes (project row.x) ---
        self.row_proj = nn.Linear(row_in_dim, hidden_channels)

        # --- Value-node embeddings ---
        self.emb = nn.ModuleDict()
        for ntype in self.node_types:
            if ntype == row_node_type:
                continue
            self.emb[ntype] = nn.Embedding(int(num_nodes_by_type[ntype]), hidden_channels)

        # --- Hetero GraphSAGE ---
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList() if use_layernorm else None

        for _ in range(num_layers):
            self.convs.append(
                HeteroConv(
                    {etype: SAGEConv((-1, -1), hidden_channels, aggr=sage_aggr) for etype in self.edge_types},
                    aggr="sum",
                )
            )
            if use_layernorm:
                self.norms.append(nn.ModuleDict({nt: nn.LayerNorm(hidden_channels) for nt in self.node_types}))

        # --- Fusion gate: decides how much graph vs wide signal ---
        # gate in (0,1): fused = wide + gate * graph
        self.gate = nn.Sequential(
            nn.Linear(2 * hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 1),
        )


        # make initial gate ~ small (sigmoid(-2) ~ 0.12)
        nn.init.constant_(self.gate[-1].bias, -2.0)

        # --- Head ---
        self.head = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, 1),
        )

        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
        for e in self.emb.values():
            nn.init.normal_(e.weight, mean=0.0, std=0.02)
        for conv in self.convs:
            for c in conv.convs.values():
                c.reset_parameters()

    def _init_x_dict(self, data: HeteroData) -> Dict[str, torch.Tensor]:
        device = next(self.parameters()).device
        x_dict: Dict[str, torch.Tensor] = {}

        row_x = data[self.row_node_type].x.to(device)
        x_dict[self.row_node_type] = self.row_proj(row_x)

        for ntype in self.node_types:
            if ntype == self.row_node_type:
                continue
            n = int(data[ntype].num_nodes)
            if hasattr(data[ntype], "n_id") and data[ntype].n_id is not None:
                idx = data[ntype].n_id.to(device)  # global ids
            else:
                idx = torch.arange(n, device=device)
            x_dict[ntype] = self.emb[ntype](idx)

        return x_dict

    def forward(self, data: HeteroData) -> torch.Tensor:
        device = next(self.parameters()).device
        data = data.to(device)

        # Wide path representation (row-only)
        row_x = data[self.row_node_type].x
        h_wide = self.row_mlp(row_x)

        # Graph path representation
        x_dict = self._init_x_dict(data)
        edge_index_dict = data.edge_index_dict

        for li, conv in enumerate(self.convs):
            x_dict = conv(x_dict, edge_index_dict)
            if li < self.num_layers - 1:
                for ntype in x_dict:
                    if self.norms is not None:
                        x_dict[ntype] = self.norms[li][ntype](x_dict[ntype])
                    x_dict[ntype] = F.relu(x_dict[ntype])
                    x_dict[ntype] = F.dropout(x_dict[ntype], p=self.dropout, training=self.training)

        h_graph = x_dict[self.row_node_type]

        # Fuse (wide + gated graph)
        gate = torch.sigmoid(self.gate(torch.cat([h_wide, h_graph], dim=-1)))  # [N,1]
        h = h_wide + gate * h_graph

        logits = self.head(h).view(-1)
        return logits

    @torch.no_grad()
    def embed_rows(self, data, device, which: str = "fused"):
        """
        Returns row embeddings aligned to row order in `data`.
        which in {"wide", "graph", "fused"}.
        """
        self.eval()
        d = data.to(device)

        row_x = d["row"].x
        h_wide = self.row_mlp(row_x)

        x_dict = self._init_x_dict(d)
        edge_index_dict = d.edge_index_dict
        for li, conv in enumerate(self.convs):
            x_dict = conv(x_dict, edge_index_dict)
            if li < self.num_layers - 1:
                for ntype in x_dict:
                    if self.norms is not None:
                        x_dict[ntype] = self.norms[li][ntype]
                    x_dict[ntype] = F.relu(x_dict[ntype])
                    x_dict[ntype] = F.dropout(x_dict[ntype], p=self.dropout, training=False)

        h_graph = x_dict["row"]
        gate = torch.sigmoid(self.gate(torch.cat([h_wide, h_graph], dim=-1)))
        h_fused = h_wide + gate * h_graph

        if which == "wide":
            return h_wide.detach().cpu()
        if which == "graph":
            return h_graph.detach().cpu()
        if which == "fused":
            return h_fused.detach().cpu()
        raise ValueError("which must be one of: wide, graph, fused")

In [ ]:
import torch
from torch_geometric.loader import NeighborLoader

def make_train_loader(
    data_train,
    *,
    batch_size: int = 1024,
    num_neighbors=(25,),   # start 1-hop; later try (25,10)
    num_workers: int = 0,
):
    train_idx = torch.arange(data_train["row"].num_nodes)
    return NeighborLoader(
        data_train,
        input_nodes=("row", train_idx),
        num_neighbors=list(num_neighbors),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score

def roc_auc_safe(y: np.ndarray, p: np.ndarray) -> float:
    return float("nan") if len(np.unique(y)) < 2 else float(roc_auc_score(y, p))

@torch.no_grad()
def eval_fullbatch_probs(model, data, device):
    model.eval()
    d = data.to(device)
    logits = model(d).view(-1)
    probs = torch.sigmoid(logits).detach().cpu().numpy()
    y = d["row"].y.view(-1).detach().cpu().numpy().astype(np.int32)
    return probs, y

@torch.no_grad()
def eval_fullbatch_auc(model, data, device):
    p, y = eval_fullbatch_probs(model, data, device)
    return roc_auc_safe(y, p)

def best_f1_threshold(y: np.ndarray, p: np.ndarray, grid: int = 200):
    ts = np.linspace(0.01, 0.99, grid)
    best_t, best_f1 = 0.5, -1.0
    for t in ts:
        f1 = f1_score(y, (p >= t).astype(np.int32))
        if f1 > best_f1:
            best_f1, best_t = float(f1), float(t)
    return best_t, best_f1

In [ ]:
import copy
import torch
import torch.nn as nn

def train_widedeep_neighbors_best_auc(
    model,
    *,
    data_train,
    data_val,
    train_loader,
    device,
    lr: float = 3e-4,
    weight_decay: float = 1e-4,
    epochs: int = 50,
    patience: int = 10,
    grad_clip: float | None = 1.0,
    print_every: int = 1,
):
    model = model.to(device)

    # pos_weight from full train labels (recommended)
    y_train = data_train["row"].y.detach().cpu().numpy().astype(np.int32)
    pos = float((y_train == 1).sum())
    neg = float((y_train == 0).sum())
    pos_weight = neg / max(pos, 1.0)

    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], device=device))
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.5,
    patience=2,
    threshold=1e-4,
    min_lr=1e-6,
    verbose=True,)

    best = {"val_auc": -1.0, "epoch": -1, "state": None}
    bad = 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, total_seeds = 0.0, 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            logits = model(batch).view(-1)
            bs = int(batch["row"].batch_size)  # seed rows
            logits_seed = logits[:bs]
            y_seed = batch["row"].y[:bs].view(-1).float()

            loss = criterion(logits_seed, y_seed)
            loss.backward()

            if grad_clip is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            optimizer.step()

            total_loss += float(loss.item()) * bs
            total_seeds += bs

        avg_loss = total_loss / max(total_seeds, 1)

        # Full-batch validation AUC (stable)
        val_auc = eval_fullbatch_auc(model, data_val, device)

        improved = val_auc > best["val_auc"] + 1e-6
        if improved:
            best["val_auc"] = float(val_auc)
            best["epoch"] = epoch
            best["state"] = copy.deepcopy({k: v.detach().cpu() for k, v in model.state_dict().items()})
            bad = 0
        else:
            bad += 1

        if epoch % print_every == 0:
            print(f"Epoch {epoch:03d} | loss={avg_loss:.4f} | val_auc={val_auc:.4f} | bad={bad}/{patience}")

        if bad >= patience:
            break

    if best["state"] is not None:
        model.load_state_dict(best["state"])

    # Report best checkpoint + compute best-F1 threshold once
    val_probs, val_y = eval_fullbatch_probs(model, data_val, device)
    thr_star, val_f1 = best_f1_threshold(val_y, val_probs, grid=200)

    report = {
        "best_epoch": best["epoch"],
        "best_val_auc": best["val_auc"],
        "val_f1_at_thr_star": float(val_f1),
        "thr_star_from_val": float(thr_star),
        "pos_weight": float(pos_weight),
    }
    return model, report

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

metadata = data_train.metadata()
num_nodes_by_type = {nt: int(data_train[nt].num_nodes) for nt in metadata[0] if nt != "row"}

model = WideDeepHeteroSAGE(
    metadata=metadata,
    num_nodes_by_type=num_nodes_by_type,
    row_in_dim=305,          # your row.x dim
    hidden_channels=64,
    num_layers=1,            # start 1-hop
    dropout=0.3,
    sage_aggr="mean",
    mlp_hidden=256,
)

In [ ]:
train_loader = make_train_loader(
    data_train,
    batch_size=1024,
    num_neighbors=(25,),     # 1-hop
    num_workers=0,
)

In [ ]:
model, report = train_widedeep_neighbors_best_auc(
    model,
    data_train=data_train,
    data_val=data_val,
    train_loader=train_loader,
    device=device,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=50,
    patience=10,
)
print(report)

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 001 | loss=0.4991 | val_auc=0.6315 | bad=0/10
Epoch 002 | loss=0.4876 | val_auc=0.6319 | bad=0/10
Epoch 003 | loss=0.4829 | val_auc=0.6376 | bad=0/10
Epoch 004 | loss=0.4803 | val_auc=0.6402 | bad=0/10
Epoch 005 | loss=0.4778 | val_auc=0.6374 | bad=1/10
Epoch 006 | loss=0.4753 | val_auc=0.6438 | bad=0/10
Epoch 007 | loss=0.4745 | val_auc=0.6463 | bad=0/10
Epoch 008 | loss=0.4712 | val_auc=0.6524 | bad=0/10
Epoch 009 | loss=0.4703 | val_auc=0.6568 | bad=0/10
Epoch 010 | loss=0.4678 | val_auc=0.6588 | bad=0/10
Epoch 011 | loss=0.4677 | val_auc=0.6631 | bad=0/10
Epoch 012 | loss=0.4648 | val_auc=0.6646 | bad=0/10
Epoch 013 | loss=0.4630 | val_auc=0.6645 | bad=1/10
Epoch 014 | loss=0.4611 | val_auc=0.6617 | bad=2/10
Epoch 015 | loss=0.4615 | val_auc=0.6654 | bad=0/10
Epoch 016 | loss=0.4612 | val_auc=0.6716 | bad=0/10
Epoch 017 | loss=0.4600 | val_auc=0.6700 | bad=1/10
Epoch 018 | loss=0.4579 | val_auc=0.6739 | bad=0/10
Epoch 019 | loss=0.4541 | val_auc=0.6714 | bad=1/10
Epoch 020 | 

In [ ]:
@torch.no_grad()
def predict_test_probs(model, data_test, device):
  model.eval()
  d = data_test.to(device)
  logits = model(d).view(-1)
  probs = torch.sigmoid(logits).cpu().numpy()
  return probs

In [ ]:
test_probs = predict_test_probs(model, data_test, device)
print(test_probs.shape)   # (num_test_rows,)

(825,)


In [ ]:
task.evaluate(test_probs) # just GNN

{'average_precision': np.float64(0.7761568328505457),
 'accuracy': 0.6339393939393939,
 'f1': 0.6447058823529411,
 'roc_auc': np.float64(0.7167919799498748)}

# Fuse embeddings with tabular

In [ ]:
def add_embeddings(df: pd.DataFrame, emb: "np.ndarray", prefix: str = "gnn_") -> pd.DataFrame:
    emb_df = pd.DataFrame(emb, columns=[f"{prefix}{i}" for i in range(emb.shape[1])])
    return pd.concat([df.reset_index(drop=True), emb_df], axis=1)

df_train_aug = add_embeddings(df_train, emb_train)
df_val_aug   = add_embeddings(df_val,   emb_val)
df_test_aug  = add_embeddings(df_test,  emb_test)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor


path = "/Models/"

predictor = TabularPredictor(
    label="outcome",
    path=path,
    eval_metric="roc_auc",
).fit(
    train_data=df_train_aug,
    tuning_data=df_val_aug,
    time_limit=3600,
    presets="medium_quality",   # medium is enough to beat RDL, RGT
    included_model_types=[
        "GBM",      # LightGBM
        "CAT",      # CatBoost
        "XGB",      # XGBoost
        "RF",       # optional
        "XT",       # optional
        "REALMLP",  # MLP (if available)
        # optionally: "NN_TORCH" (depending on your AG version/install)
    ],
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.4.1+cu121
CUDA Version:       12.1
GPU Memory:         GPU 0: 14.70/14.74 GB
Total GPU Memory:   Free: 14.70 GB, Allocated: 0.04 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       7.19 GB / 12.67 GB (56.8%)
Disk Space Avail:   173.27 GB / 235.68 GB (73.5%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/content/drive/MyDrive/ICML-transactions/trials_data/Models/Tabular/tabular+gnn(T)"
Train Data Rows:    11994
Train Data Columns: 94
Tuning Data Rows:    960
Tuning Data Columns: 94
Label Column:       outcome
AutoGluon infers your prediction problem is: 'binary' 

In [ ]:
print(predictor.leaderboard(df_val_aug, silent=True).head(20))

                 model  score_test  score_val eval_metric  pred_time_test  \
0  WeightedEnsemble_L2    0.701129   0.701129     roc_auc        0.939949   
1             CatBoost    0.694276   0.694276     roc_auc        0.115787   
2     RandomForestEntr    0.693918   0.693918     roc_auc        0.307346   
3     RandomForestGini    0.692134   0.692134     roc_auc        0.352164   
4           LightGBMXT    0.692024   0.692024     roc_auc        0.108782   
5             LightGBM    0.690157   0.690157     roc_auc        0.100283   
6       ExtraTreesEntr    0.689819   0.689819     roc_auc        0.402246   
7        LightGBMLarge    0.686471   0.686471     roc_auc        0.086230   
8              XGBoost    0.678065   0.678065     roc_auc        0.105120   
9       ExtraTreesGini    0.674067   0.674067     roc_auc        0.391507   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.495127  215.847269                 0.005788                0.

In [ ]:
proba  = predictor.predict_proba(df_test_aug)
preds_proba = proba[1]

results = task.evaluate(preds_proba)

{'average_precision': np.float64(0.7800480047352947),
 'accuracy': 0.6727272727272727,
 'f1': 0.755877034358047,
 'roc_auc': np.float64(0.724238131560786)}

In [ ]:
OUTPUT_PATH = '/runs/'

df_tab_GNN_T = pd.DataFrame([results])
df_tab_GNN_T["model"] = "Tab+GNN(T)"
df_tab_GNN_T["task"] = task

df_tab_GNN_T.to_csv(OUTPUT_PATH+f"Tab-GNN(T)_{task}.csv")